In [ ]:
import warnings

warnings.filterwarnings("ignore")
import os

if "jbook" in os.getcwd():
    os.chdir(os.path.abspath(os.path.join("../..")))
FORCE = False

# AppVoCAI Dataset Enrichment
This data enrichment effort will imbue subsequent data quality and exploratory analyses with essential quality signals, user engagement data, target class distributions, and aggregations that position us for a systematic, and intensive data quality analysis, and an insight-rich exploratory effort. This data enrichment stage will unfold through five progressive steps:

1. **Sentiment Classification**: Leveraging SpaCy's rule-based sentiment classifier will provide a computationally efficient, high-level sense of sentiment distribution and balance in the dataset. 
2. **Text Quality Assessment**: We evaluate the grammatical sophistication, syntactic structure, diversity, coherence, clarity, intensity, and linguistic elaborateness of the user reviews - aspects known to sizeably effect LLM model performance.
3. **Quantitative Enrichment**: Disembling timestamps will release temporal features, such as the age of each review relative to the most recent one and the month, day, and hour of submission for temporal and longitudinal analysis of user engagement patterns of app usage and cyclical trends in user behavior. Deviations from broader themes at the category and app levels may reveal feature gaps, unmet needs, and notable variability in user experiences.
4. **Aggregate Data**: Summarizing information at the app, author, and category levels to expose overarching themes related to user engagement, satisfaction, and app performance, providing a macro-level perspective on the data.

Is this early feature engineering? Great question. It is not. Feature engineering derives new variables that are expected to have an influential effect on model development and predictive performance. While there may be some overlap, our purpose is to facilitate rigorous data quality analysis, and exploration while minimizing bias, and avoiding transformations that might distort or invalidate analytical interpretations.

Let's do this.


## Import Libraries

In [ ]:
from discover.container import DiscoverContainer
from discover.infra.config.flow import FlowConfigReader
from discover.core.flow import DataPrepStageDef
from discover.flow.data_prep.quant.stage import QuantStage
from discover.flow.data_prep.sentiment.stage import SentimentClassificationStage
from discover.flow.data_prep.tqa.stage import TQAStage
from discover.flow.data_prep.aggregation.stage import AggregationStage
from discover.core.flow import PhaseDef, DataPrepStageDef

## Dependency Container

In [ ]:
container = DiscoverContainer()
container.init_resources()
container.wire(
    modules=[
        "discover.flow.data_prep.stage",
        "discover.flow.data_prep.aggregation.stage",
    ],
)

## Sentiment Classification Pipeline  
The Review-Level Sentiment Classification Pipeline uses spaCy to analyze sentiment on a scale from -1 to 1. Reviews are then classified as negative, neutral, or positive by dividing this scale into three equal spans.

In [ ]:
# Obtain the configuration
reader = FlowConfigReader()
stage_config = reader.get_stage_config(
    phase=PhaseDef.DATAPREP, stage=DataPrepStageDef.SENTIMENT
)

# Build and run Data Ingestion Stage
stage = SentimentClassificationStage.build(stage_config=stage_config, force=FORCE)
asset_id = stage.run()

---

## Review Text Quality Analysis (TQA) Pipeline
Review text quality is an indicator of the content's richness, coherence, and informativeness. In this section, we integrate two complementary quality assessment measures—a lexical/syntactic score and a perplexity-based score—into a weighted sum. This approach provides a balanced evaluation, capturing both the structural diversity and natural language fluency of the reviews.

### Lexical and Syntactic Complexity Assessment  
The lexical and syntactic quality assessment (TQA) evaluates review quality using a composite score derived from multiple syntactic and lexical measures. These measures are computed with specific weights:

1. **POS Count Score (40%)**: Reflects the richness of content using counts of nouns, verbs, adjectives, and adverbs.
2. **POS Diversity Score (20%)**: Captures variety in language using an entropy-based calculation.
3. **POS Intensity Score (10%)**: Assesses the density of key parts of speech relative to total word count.
4. **Structural Complexity Score (20%)**: Evaluates text complexity using unique word proportion, special character usage, and word length variation.
5. **TQA Check Score (10%)**: Incorporates quality signals such as limited digit use, minimal special characters, and proper terminal punctuation.

A high Lexical and Syntactic Complexity Score typically indicates a text rich in linguistic features, with varied sentence structures and a well-balanced mix of nouns, verbs, and modifiers (like adjectives and adverbs). This variety is particularly valuable for tasks like Aspect-Based Sentiment Analysis (ABSA), where structural complexity can signal content with nuanced aspects and sentiments.

### Perplexity-Based Quality Assessment  
This measure evaluates review quality by applying 13 linguistic and structural filters, each assigned a weight derived from relative perplexity differences between the full dataset and filtered subsets. The filters assess features like adjective presence, punctuation ratios, word repetition, and special character use. Weights are computed to emphasize filters that most reduce perplexity, thus enhancing text fluency and coherence. The final score is a weighted sum of these filter indicators. 

Lower perplexity implies higher fluency, coherence, and grammatical correctness, which are key indicators of text quality. This component is useful for flagging low-quality or noisy text that may be unpredictable or deviate significantly from standard linguistic norms.

### Weighted Scoring Approach
To create a balanced quality score, the Syntactic Complexity Score and Perplexity-Based Score are combined with tailored weights that emphasize their respective strengths. 

- **Syntactic Complexity Weight**: Typically given more weight when the task demands detailed and linguistically rich text, such as ABSA, where richer syntactic content improves aspect and sentiment extraction.
- **Perplexity-Based Weight**: Often assigned a moderate weight to capture coherence and fluency, ensuring that only grammatically sound and predictable text is prioritized without sacrificing syntactic diversity.

The final **Text Quality Score** is a weighted average of these two components, providing a single score that balances both syntactic richness and linguistic fluency. The remainder of this notebook will execute the text quality scoring pipeline, computing and integrating the two quality measures into a final text quality score. The specifics of the measures are provided in {ref}`Text Quality Method` Section of the appendix.   

In [ ]:
# Obtain the configuration
stage_config = reader.get_stage_config(
    phase=PhaseDef.DATAPREP, stage=DataPrepStageDef.TQA
)

# Build and run Data Ingestion Stage
stage = TQAStage.build(stage_config=stage_config, force=FORCE)
asset_id = stage.run()

## Quantitative Enrichment
This pipeline enriches the dataset by extracting and analyzing key features such as **review age**, **review length**, and three **temporal fields**—**month**, **day of the week**, and **hour of the day**. These features are designed to provide insights into user engagement patterns and app usage, supporting unmet needs discovery in the following ways:

1. **Review Age**: Understanding how review content changes over time can help identify shifting user expectations and long-standing issues that may require attention. Analyzing review age trends can inform decisions on product updates and feature prioritization.

2. **Review Length**: Longer reviews often contain richer, more detailed feedback. By evaluating review length, we can identify opportunities to address comprehensive user concerns or highlight areas where users have strong opinions or pain points.

3. **Month**: Monthly trends can reveal **seasonal usage patterns**, indicating how user needs change throughout the year. This can inform strategies like seasonal feature releases, marketing campaigns, or resource allocation.

4. **Day of the Week**: Analyzing reviews by the day of the week might uncover insights into **weekly routines** and how apps fit into users' schedules. For instance, productivity apps might show spikes on weekdays, while entertainment apps might peak on weekends, guiding feature development aligned with these usage patterns.

5. **Hour of the Day**: Understanding when users are most active can provide clues about **contextual needs**. Apps with high engagement at night might suggest opportunities for sleep-related features, while those used during commutes may benefit from quick, on-the-go functionalities.

Together, these metadata features enhance our understanding of app usage and user behavior, laying the foundation for identifying unmet needs and informing opportunity discovery.

In [ ]:
stage_config = reader.get_stage_config(
    phase=PhaseDef.DATAPREP, stage=DataPrepStageDef.QUANT
)

# Build and run Data Ingestion Stage
stage = QuantStage.build(stage_config=stage_config, force=FORCE)
asset_id = stage.run()

## Aggregation Pipelines
Aggregating data at the app and category levels provides a high-level view of review trends and user behavior, offering insights into user engagement and feedback patterns. At the app level, we consolidate key metrics, such as average ratings, review length, review count, and total vote sum, while identifying standout reviews based on highest vote counts, top TQA scores, and longest review lengths. 

A similar approach is used at the category level, aggregating metrics across all apps within a category to reveal trends that may indicate common strengths or pain points across similar apps. This two-tiered aggregation—app-level and category-level—allows for both detailed and broad insights into app performance, aiding in strategic decisions and market comparisons.

### App Aggregation Pipeline

In [ ]:
FORCE = True
# Obtain the configuration
stage_config = reader.get_stage_config(
    phase=PhaseDef.DATAPREP, stage=DataPrepStageDef.AGG
)

# Build and run Data Ingestion Stage
stage = AggregationStage.build(stage_config=stage_config, force=FORCE)
asset_id = stage.run()

## Enrichment Stage Wrap-Up
The enrichment stage enhanced the dataset with features, including review metadata (such as length, age and temporal data), sentiment analysis, text quality scores, and comprehensive app- and category-level aggregations. In the upcoming EDA phase, we will leverage these enriched attributes to uncover patterns, relationships, and trends that illuminate user behavior and app performance.